# Lane Following - Data collection



### Import library module
Import all the libraries needed for Data Collection.

Using OpenCV to visualize and save images with labels. Uuid, datetime and other libraries are used for image naming.

In [ ]:
# IPython libraries be used to display and parts
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
# Camera and servo interface foe Jetbot robot car
from jetbot import Robot, Camera, bgr8_to_jpeg
# Python base package for image annotations
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time
from servoserial import ServoSerial
import threading


# Display camera data in real time

The neural network takes an image of 224x224 pixels as input.



In [ ]:
camera = Camera()

image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')

def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

display(widgets.HBox([image_widget, target_widget]), x_slider, y_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0, step=0.001)

FloatSlider(value=0.0, description='y', max=1.0, min=-1.0, step=0.001)

# Create a handle controller instance




We will use this index to create and display the controller.

In [ ]:
controller = widgets.Controller(index=0)
robot = Robot()
display(controller)

Controller()

### Connect the controller to the label image.

In [ ]:
widgets.jsdlink((controller.axes[2], 'value'), (x_slider, 'value'))
widgets.jsdlink((controller.axes[3], 'value'), (y_slider, 'value'))

# Data collection

The code below will display the live image feed and the number of images we saved.

We store the value of the target X, Y:

1. Put the green dot on the target
2. Press the 13th button to save

Then the data we want will be saved to the ``dataset_xy`` folder. The saved file naming format is:
``xy_<x value>_<y value>_<uuid>.jpg``
When we train, we load the image and parse the x and y values in the file name.

In [ ]:
DATASET_DIR = 'dataset_xy'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

for b in controller.buttons:
    b.unobserve_all()

count_widget = widgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))

def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, uuid1())

def save_snapshot(change):
    if change['new']:
        uuid = xy_uuid(x_slider.value, y_slider.value)
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))


controller.buttons[4].observe(save_snapshot, names='value')

display(widgets.VBox([
    target_widget,
    count_widget
]))

Create a method to adjust the position of the servo to get the lane following angle and call this method to adjust.

In [ ]:
servo_device = ServoSerial()
def camservoInitFunction():
    global leftrightpulse, updownpulse
    leftrightpulse = 2048
    updownpulse = 2048
    servo_device.Servo_serial_control(1, 2048)
    time.sleep(0.1)
    servo_device.Servo_serial_control(2, 1300)

Call the below method to adjust the pan/tilt to the lane following angle

In [ ]:
camservoInitFunction()